# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [3]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pathlib import Path

# Import API key
from api_keys import geoapify_key

In [5]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file = Path("../output_data/cities.csv")
city_data_df = pd.read_csv(csv_file)

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,ebon,4.5820,168.6964,28.61,76,100,6.49,MH,1693004334
1,1,phulbani,20.4667,84.2333,21.89,93,91,1.82,IN,1693004334
2,2,humberto de campos,-2.5983,-43.4611,25.07,87,9,2.94,BR,1693004334
3,3,port alfred,-33.5906,26.8910,11.21,88,59,2.68,ZA,1693004335
4,4,mount gambier,-37.8333,140.7667,9.20,71,89,2.95,AU,1693004335


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [8]:
%%capture --no-display

# Configure the map plot
cities_map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    size = "Humidity",
    scale = 1,
    color = "City"
)

# Display the map
cities_map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [10]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_cities=city_data_df.loc[(city_data_df['Max Temp']<32)&(city_data_df['Max Temp']>15)&
                             (city_data_df['Humidity']<80)&(city_data_df['Humidity']>40)&
                              (city_data_df['Wind Speed']<8)&(city_data_df['Cloudiness']>30)]

# Drop any rows with null values
clean_ideal_cities=ideal_cities.dropna()

# Display sample data
clean_ideal_cities

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,ebon,4.5820,168.6964,28.61,76,100,6.49,MH,1693004334
15,15,barra de santo antonio,-9.4047,-35.5072,22.60,79,95,4.21,BR,1693004059
22,22,utrik,11.2278,169.8474,28.44,71,100,6.86,MH,1693004338
23,23,tandalti,13.0167,31.8667,29.96,47,100,5.43,SD,1693004338
30,30,hithadhoo,-0.6000,73.0833,28.05,77,40,5.40,MV,1693004339
...,...,...,...,...,...,...,...,...,...,...
536,536,suva,-18.1416,178.4415,24.16,67,75,1.46,FJ,1693004458
540,540,kalmunai,7.4167,81.8167,26.35,79,90,3.58,LK,1693004459
544,544,dhidhdhoo,6.8833,73.1000,27.75,72,82,1.45,MV,1693004460
556,556,saint-joseph,-21.3667,55.6167,19.21,75,39,4.82,RE,1693004463


### Step 3: Create a new DataFrame called `hotel_df`.

In [13]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = clean_ideal_cities.loc[:,['City', 'Country', 'Lat', 'Lng', 'Humidity']]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
0,ebon,MH,4.5820,168.6964,76,
15,barra de santo antonio,BR,-9.4047,-35.5072,79,
22,utrik,MH,11.2278,169.8474,71,
23,tandalti,SD,13.0167,31.8667,47,
30,hithadhoo,MV,-0.6000,73.0833,77,
...,...,...,...,...,...,...
536,suva,FJ,-18.1416,178.4415,67,
540,kalmunai,LK,7.4167,81.8167,79,
544,dhidhdhoo,MV,6.8833,73.1000,72,
556,saint-joseph,RE,-21.3667,55.6167,75,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [14]:
# Set base URL
base_url = "https://api.geoapify.com/v2/places"
# Set parameters to search for a hotel
radius = 10000
categories = "accomodation.hotel"
limit = 5
params = {
    "categories":categories,
    "limit":limit,
    "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude=row['Lat']
    longitude=row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    
   # Make and API request using the params dictionaty, Convert the API response to JSON format
    name_address = requests.get(base_url, params=params).json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
ebon - nearest hotel: No hotel found
barra de santo antonio - nearest hotel: No hotel found
utrik - nearest hotel: No hotel found
tandalti - nearest hotel: No hotel found
hithadhoo - nearest hotel: No hotel found
la ronge - nearest hotel: No hotel found
uturoa - nearest hotel: No hotel found
puerto baquerizo moreno - nearest hotel: No hotel found
cockburn town - nearest hotel: No hotel found
san andres - nearest hotel: No hotel found
yellowknife - nearest hotel: No hotel found
pyetrykaw - nearest hotel: No hotel found
atafu village - nearest hotel: No hotel found
weno - nearest hotel: No hotel found
happy valley-goose bay - nearest hotel: No hotel found
nantucket - nearest hotel: No hotel found
rivne - nearest hotel: No hotel found
fale old settlement - nearest hotel: No hotel found
whitehorse - nearest hotel: No hotel found
caraballeda - nearest hotel: No hotel found
verkhnevilyuysk - nearest hotel: No hotel found
kavaratti - nearest hotel: No hotel found
gaigetu

,City,Country,Lat,Lng,Humidity,Hotel Name
0,ebon,MH,4.5820,168.6964,76,No hotel found
15,barra de santo antonio,BR,-9.4047,-35.5072,79,No hotel found
22,utrik,MH,11.2278,169.8474,71,No hotel found
23,tandalti,SD,13.0167,31.8667,47,No hotel found
30,hithadhoo,MV,-0.6000,73.0833,77,No hotel found
...,...,...,...,...,...,...
536,suva,FJ,-18.1416,178.4415,67,No hotel found
540,kalmunai,LK,7.4167,81.8167,79,No hotel found
544,dhidhdhoo,MV,6.8833,73.1000,72,No hotel found
556,saint-joseph,RE,-21.3667,55.6167,75,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [15]:
%%capture --no-display

# Configure the map plot
hotel_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    size = "Humidity",
    scale = 1,
    color = "City",
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map plot
hotel_plot


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)